In [ ]:
#TradingView
#TradingEconomics
#Forexfactory
#SeekingAlpha.com
#Investing.com
#Nasdaq Earnings Calendar
#Yahoo Finance
#Zacks.com
#
#
#
#
#
#


In [1]:
from datetime import date, timedelta
import pandas as pd
import requests
from pandas import DataFrame

def request(url: str, method: str = "get", timeout: int = 0, **kwargs) -> requests.Response:
    method = method.lower()
    if method not in ["delete", "get", "head", "patch", "post", "put"]:
        raise ValueError(f"Invalid method: {method}")
    headers = kwargs.pop("headers", {})
    headers["User-Agent"] = headers.get("User-Agent", 
                                        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                                        "(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    timeout = timeout or 10
    func = getattr(requests, method)
    return func(url, headers=headers, timeout=timeout, **kwargs)

def get_filters(date_str: str) -> str:
    return f"?filter[selected_date]={date_str}&filter[with_rating]=false&filter[currency]=USD"

class EventsFetcher:
    def __init__(self):
        pass
    
    @staticmethod
    def get_next_earnings(limit: int = 5, start_date: date = date.today()) -> DataFrame:
        base_url = "https://seekingalpha.com/api/v3/earnings_calendar/tickers"
        df_earnings = pd.DataFrame()
        
        for _ in range(0, limit):
            start_date = pd.to_datetime(start_date)
            date_str = str(start_date.strftime("%Y-%m-%d"))
            response = request(base_url + get_filters(date_str), timeout=10)
            json = response.json()
            
            try:
                data = json["data"]
                cleaned_data = [x["attributes"] for x in data]
                temp_df = pd.DataFrame.from_records(cleaned_data)
                temp_df = temp_df.drop(columns=["sector_id"])
                temp_df["Date"] = start_date  # pylint: disable=E1137
                df_earnings = pd.concat([df_earnings, temp_df], join="outer", ignore_index=True)
                start_date = start_date + timedelta(days=1)
            except KeyError:
                pass
        
        df_earnings = df_earnings.rename(
            columns={
                "slug": "Ticker",
                "name": "Name",
                "release_time": "Release Time",
                "exchange": "Exchange",
            }
        )
        
        if df_earnings.empty:
            print("No earnings found. Try adjusting the date.\n")
            return pd.DataFrame()
        
        df_earnings = df_earnings[df_earnings["Date"] <= pd.to_datetime(start_date + timedelta(days=limit))]
        return df_earnings

if __name__ == "__main__":
    events_fetcher_ = EventsFetcher()
    upcoming_events_ = events_fetcher_.get_next_earnings()
    print(upcoming_events_)

    Ticker                       Name Exchange Release Time  \
0      WDS  Woodside Energy Group Ltd     NYSE   Pre-Market   
1        S          SentinelOne, Inc.     NYSE  Post-Market   
2      PVH                  PVH Corp.     NYSE  Post-Market   
3      BOX                  Box, Inc.     NYSE  Post-Market   
4     NCNO                nCino, Inc.   NASDAQ  Post-Market   
..     ...                        ...      ...          ...   
239    MOB          Mobilicom Limited   NASDAQ   Pre-Market   
240   CTHR    Charles & Colvard, Ltd.   NASDAQ  Post-Market   
241   ONCO            Onconetix, Inc.   NASDAQ  Post-Market   
242   LGHL    Lion Group Holding Ltd.   NASDAQ    Estimated   
243   SPRC              SciSparc Ltd.   NASDAQ  Post-Market   

                 release_date     marketcap       Date  
0    2024-08-27T04:00:00.000Z  3.392646e+10 2024-08-27  
1    2024-08-27T04:00:00.000Z  7.787382e+09 2024-08-27  
2    2024-08-27T04:00:00.000Z  5.791894e+09 2024-08-27  
3    2024-08-27

In [3]:
import pandas as pd

# Assuming 'upcoming_events_' is your DataFrame

# Group by 'Date' and then further by 'Release Time'
grouped_by_date = upcoming_events_.groupby('Date')

# Define the release time categories
release_time_categories = ["Pre-Market", "During-Market", "Post-Market", "Estimated"]

# Iterate through each date group and print the companies by release time categories
for date, group in grouped_by_date:
    print(f"Date: {date}")
    for category in release_time_categories:
        # Filter companies that fall under the current release time category
        companies_in_category = group[group['Release Time'] == category]['Name'].tolist()
        if companies_in_category:
            print(f"  {category}: {', '.join(companies_in_category)}")
    print("\n")  # Separate each date's output with a newline

Date: 2024-08-27 00:00:00
  Pre-Market: Woodside Energy Group Ltd, American Woodmark Corporation, ScanSource, Inc., Gaotu Techedu Inc., Super Hi International Holding Ltd., REX American Resources Corporation, The Hain Celestial Group, Inc., Jiayin Group Inc., Everest Consolidator Acquisition Corporation, Citi Trends, Inc., Allot Ltd., Imperial Petroleum Inc., Perception Capital Corp. III, 99 Acquisition Group Inc., CNFinance Holdings Limited, Alpine 4 Holdings, Inc.
  During-Market: Cadeler A/S, Zalatoris Acquisition Corp., Tupperware Brands Corporation, Qomolangma Acquisition Corp., urban-gro, Inc., SPI Energy Co., Ltd.
  Post-Market: SentinelOne, Inc., PVH Corp., Box, Inc., nCino, Inc., Nordstrom, Inc., Semtech Corporation, Ambarella, Inc., JOYY Inc., TWFG, Inc., VNET Group, Inc., Oculis Holding AG, Waldencast plc, LexinFintech Holdings Ltd., EVe Mobility Acquisition Corp, Electromed, Inc., Athena Technology Acquisition Corp. II, Koss Corporation, OneMedNet Corporation, Gulf Resource

In [19]:
import pandas as pd
import requests

url = 'https://economic-calendar.tradingview.com/events'
today = pd.Timestamp.today().normalize()
headers = {
    'Origin': 'https://in.tradingview.com'
}
payload = {
    'from': (today + pd.offsets.Hour(23)).isoformat() + '.000Z',
    'to': (today + pd.offsets.Day(7) + pd.offsets.Hour(22)).isoformat() + '.000Z',
    'countries': ','.join(['US', 'IN', "FR", "EU"]),
    "minImportance":"-1" #-1 , 0 , 1 
}
data = requests.get(url, headers=headers, params=payload).json()
df = pd.DataFrame(data['result'])

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page you want to scrape
url = "https://tradingview.com/markets/stocks-usa/earnings/"

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Parse the page content
soup = BeautifulSoup(response.content, "html.parser")

# Find the table or specific content you need
# Assuming the table is within a div with class `tv-screener__content-pane`
table = soup.find("div", class_="tv-screener__content-pane")

# Extract headers
visible_columns = table.find_all('thead th:not([class*=i-hidden])')
header = [col.get_text().split('\n')[0] for col in visible_columns]

# Extract rows
rows = table.find_all("tbody tr")

columns = []
for row in rows:
    column = [col.get_text().replace('\n', ' - ') for col in row.find_all('td')]
    columns.append(column)

# Create a DataFrame
df = pd.DataFrame(columns=header, data=columns)

# Display the DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


In [27]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
# pd.set_option('display.max_rows', 50000)
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 10000)

driver = webdriver.Chrome()
driver.get("https://tradingview.com/markets/stocks-usa/earnings/") # "v
driver.find_element(By.XPATH, "//div[.='This Week']").click()
time.sleep(5)

visible_columns = driver.find_elements(By.CSS_SELECTOR, 'div.tv-screener__content-pane thead th:not([class*=i-hidden])')
data_field = [c.get_attribute('data-field') for c in visible_columns]

header = [c.text.split('\n')[0] for c in visible_columns]
rows = driver.find_elements(By.XPATH, "//div[@class='tv-screener__content-pane']//tbody/tr")

columns = []
for field in data_field:
    column = driver.find_elements(By.XPATH, f"//div[@class='tv-screener__content-pane']//tbody/tr/td[@data-field-key='{field}']")
    columns.append([col.text.replace('\n',' - ') for col in column])

df = pd.DataFrame(dict(zip(header, columns)))
print(df)
driver.quit()

                                                TICKER       MKT CAP  EPS ESTIMATE REPORTED EPS  SURPRISE SURPRISE % REVENUE FORECAST REVENUE ACTUAL
0                           NVDAD - NVIDIA CORPORATION    3.136 TUSD       0.65USD            —         —          —      28.737 BUSD              —
1                              CRMD - SALESFORCE, INC.  253.616 BUSD       2.35USD            —         —          —       9.216 BUSD              —
2                          PDDD - PDD HOLDINGS INC. DR  127.425 BUSD       2.81USD      3.20USD   0.39USD     13.73%      13.779 BUSD    13.352 BUSD
3                       DELLD - DELL TECHNOLOGIES INC.   78.985 BUSD       1.71USD            —         —          —      24.148 BUSD              —
4                   CRWDD - CROWDSTRIKE HOLDINGS, INC.   64.628 BUSD       0.97USD            —         —          —     958.269 MUSD              —
5                     MRVLD - MARVELL TECHNOLOGY, INC.   59.995 BUSD       0.30USD            —         — 

In [44]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
# pd.set_option('display.max_rows', 50000)
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 10000)

driver = webdriver.Chrome()
driver.get("https://tradingview.com/markets/stocks-usa/earnings/") # "v
driver.find_element(By.XPATH, "//div[.='Next Week']").click()
time.sleep(5)

try:
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    data = []
    
    for row in rows:
        # Adjust the selectors based on the actual structure of the table
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) > 1:
            company = cells[0].text
            earnings_date = cells[1].text
            EPSestimate = cells[10].text
            # earnings_date = cells[1].text
            # company = cells[0].text
            # earnings_date = cells[1].text
            # company = cells[0].text
            # earnings_date = cells[1].text
            # company = cells[0].text
            # earnings_date = cells[1].text
            # company = cells[0].text
            
            data.append({
                "Company": company,
                "Earnings Date": earnings_date,
                "EPS estimate": EPSestimate 
            })
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    print(df)
    
except Exception as e:
    print(f"Error scraping data: {e}")

finally:
    driver.quit()

                                               Company Earnings Date EPS estimate
0                                 AVGOD\nBROADCOM INC.  736.309 BUSD             
1                                  CPRTD\nCOPART, INC.   51.204 BUSD             
2                            KRD\nKROGER COMPANY (THE)   38.443 BUSD             
3                                     ZSD\nZSCALER INC   29.318 BUSD             
4             HPED\nHEWLETT PACKARD ENTERPRISE COMPANY   24.694 BUSD             
5                                   IOTD\nSAMSARA INC.   22.721 BUSD             
6                             DLTRD\nDOLLAR TREE, INC.   20.304 BUSD             
7                      DKSD\nDICK'S SPORTING GOODS INC   19.148 BUSD             
8                       HRLD\nHORMEL FOODS CORPORATION    17.76 BUSD             
9                   CASYD\nCASEYS GENERAL STORES, INC.    13.66 BUSD             
10                     GWRED\nGUIDEWIRE SOFTWARE, INC.   12.173 BUSD             
11              

In [54]:
############ this code gets the Sun
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains


import pandas as pd

import requests
from bs4 import BeautifulSoup

# pd.set_option('display.max_rows', 50000)
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 10000)

driver = webdriver.Chrome()
driver.get("https://tradingview.com/markets/stocks-usa/earnings/") # "v
driver.find_element(By.XPATH, "//div[.='Next Week']").click()
time.sleep(5)

try:
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    data = []
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) == 11:
            company = cells[0].text
            earnings_date = cells[1].text
            col_3 = cells[2].text
            col_4 = cells[3].text
            col_5 = cells[4].text
            col_6 = cells[5].text
            col_7 = cells[6].text
            col_8 = cells[7].text
            col_9 = cells[8].text
            col_10 = cells[9].text
            
            # Identify the symbol in the last cell
            last_cell = cells[10]
            symbol = "Unknown"

            # Check if the last cell contains an SVG (Moon or Sun)
            svg_elements = last_cell.find_elements(By.TAG_NAME, "svg")
            if svg_elements:
                # Check for Sun or Moon based on known structures
                svg_element = svg_elements[0]
                paths = svg_element.find_elements(By.TAG_NAME, "path")
                
                if len(paths) == 1 and paths[0].get_attribute("d") == "M8.36 3.608a6.002 6.002 0":
                    symbol = "Moon"
                elif len(paths) > 1:
                    symbol = "Sun"
            else:
                # Check for Dash symbol
                span_element = last_cell.find_element(By.TAG_NAME, "span")
                if span_element.text == "—":
                    symbol = "Dash"

            data.append({
                "Company": company,
                "Earnings Date": earnings_date,
                "Col 3": col_3,
                "Col 4": col_4,
                "Col 5": col_5,
                "Col 6": col_6,
                "Col 7": col_7,
                "Col 8": col_8,
                "Col 9": col_9,
                "Col 10": col_10,
                "Symbol": symbol
            })
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    print(df)
    
except Exception as e:
    print(f"Error scraping data: {e}")

finally:
    driver.quit()


                                               Company Earnings Date     Col 3 Col 4 Col 5 Col 6         Col 7 Col 8       Col 9      Col 10   Symbol
0                                 AVGOD\nBROADCOM INC.  736.309 BUSD   1.21USD     —     —     —    12.97 BUSD     —  2024-09-05  2024-07-31  Unknown
1                                  CPRTD\nCOPART, INC.   51.204 BUSD   0.37USD     —     —     —    1.074 BUSD     —  2024-09-04  2024-07-31  Unknown
2                            KRD\nKROGER COMPANY (THE)   38.443 BUSD   0.91USD     —     —     —   34.082 BUSD     —  2024-09-06  2024-07-31  Unknown
3                                     ZSD\nZSCALER INC   29.318 BUSD   0.70USD     —     —     —  567.465 MUSD     —  2024-09-03  2024-07-31  Unknown
4             HPED\nHEWLETT PACKARD ENTERPRISE COMPANY   24.694 BUSD   0.47USD     —     —     —    7.659 BUSD     —  2024-09-04  2024-07-31  Unknown
5                                   IOTD\nSAMSARA INC.   22.721 BUSD   0.01USD     —     —     —  28

In [52]:
############ this code gets the Dash
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains


import pandas as pd

import requests
from bs4 import BeautifulSoup

# pd.set_option('display.max_rows', 50000)
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 10000)

driver = webdriver.Chrome()
driver.get("https://tradingview.com/markets/stocks-usa/earnings/") # "v
driver.find_element(By.XPATH, "//div[.='Next Week']").click()
time.sleep(5)


try:
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    data = []
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) == 11:
            company = cells[0].text
            earnings_date = cells[1].text
            col_3 = cells[2].text
            col_4 = cells[3].text
            col_5 = cells[4].text
            col_6 = cells[5].text
            col_7 = cells[6].text
            col_8 = cells[7].text
            col_9 = cells[8].text
            col_10 = cells[9].text
            
            # Identify the symbol in the last cell using XPath
            last_cell = cells[10]
            symbol = "Unknown"

            # Check for Moon (based on specific SVG path inside the cell)
            moon_xpath = ".//svg/path[contains(@d, 'M8.36 3.608a6.002 6.002')]"
            if last_cell.find_elements(By.XPATH, moon_xpath):
                symbol = "Moon"
            # Check for Sun (based on specific SVG path inside the cell)
            sun_xpath = ".//svg/path[contains(@d, 'M13.5 8.5h3M.5 8.5')]"
            if last_cell.find_elements(By.XPATH, sun_xpath):
                symbol = "Sun"
            # Check for Dash (—)
            dash_xpath = ".//span[text()='—']"
            if last_cell.find_elements(By.XPATH, dash_xpath):
                symbol = "Dash"

            data.append({
                "Company": company,
                "Earnings Date": earnings_date,
                "Col 3": col_3,
                "Col 4": col_4,
                "Col 5": col_5,
                "Col 6": col_6,
                "Col 7": col_7,
                "Col 8": col_8,
                "Col 9": col_9,
                "Col 10": col_10,
                "Symbol": symbol
            })
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    print(df)
    
except Exception as e:
    print(f"Error scraping data: {e}")

finally:
    driver.quit()

                                               Company Earnings Date     Col 3 Col 4 Col 5 Col 6         Col 7 Col 8       Col 9      Col 10   Symbol
0                                 AVGOD\nBROADCOM INC.  736.309 BUSD   1.21USD     —     —     —    12.97 BUSD     —  2024-09-05  2024-07-31  Unknown
1                                  CPRTD\nCOPART, INC.   51.204 BUSD   0.37USD     —     —     —    1.074 BUSD     —  2024-09-04  2024-07-31     Dash
2                            KRD\nKROGER COMPANY (THE)   38.443 BUSD   0.91USD     —     —     —   34.082 BUSD     —  2024-09-06  2024-07-31     Dash
3                                     ZSD\nZSCALER INC   29.318 BUSD   0.70USD     —     —     —  567.465 MUSD     —  2024-09-03  2024-07-31  Unknown
4             HPED\nHEWLETT PACKARD ENTERPRISE COMPANY   24.694 BUSD   0.47USD     —     —     —    7.659 BUSD     —  2024-09-04  2024-07-31  Unknown
5                                   IOTD\nSAMSARA INC.   22.721 BUSD   0.01USD     —     —     —  28

In [61]:
############ Final Code for market
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains


import pandas as pd

import requests
from bs4 import BeautifulSoup

# pd.set_option('display.max_rows', 50000)
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 10000)

driver = webdriver.Chrome()
driver.get("https://tradingview.com/markets/stocks-usa/earnings/") # "v
driver.find_element(By.XPATH, "//div[.='Next Week']").click()
time.sleep(5)


try:
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    data = []
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) == 11:
            company = cells[0].text
            earnings_date = cells[1].text
            col_3 = cells[2].text
            col_4 = cells[3].text
            col_5 = cells[4].text
            col_6 = cells[5].text
            col_7 = cells[6].text
            col_8 = cells[7].text
            col_9 = cells[8].text
            col_10 = cells[9].text
            
            # Identify the symbol in the last cell using XPath
            last_cell = cells[10]
            symbol = "After Market"


            # Check if the last cell contains an SVG (Moon or Sun)
            svg_elements = last_cell.find_elements(By.TAG_NAME, "svg")
            if svg_elements:
                # Check for Sun or Moon based on known structures
                svg_element = svg_elements[0]
            
                paths = svg_element.find_elements(By.TAG_NAME, "path")
                if len(paths) > 1:
                   symbol = "Before Market"

            # Check for Moon (based on specific SVG path inside the cell)
            dash_xpath = ".//span[text()='—']"
            if last_cell.find_elements(By.XPATH, dash_xpath):
                symbol = "Intraday"
            svg_elements = last_cell.find_elements(By.TAG_NAME, "svg")
            
            data.append({
                "Company": company,
                "Earnings Date": earnings_date,
                "Col 3": col_3,
                "Col 4": col_4,
                "Col 5": col_5,
                "Col 6": col_6,
                "Col 7": col_7,
                "Col 8": col_8,
                "Col 9": col_9,
                "Col 10": col_10,
                "Symbol": symbol
            })
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    print(df)
    
# except Exception as e:
#     print(f"Error scraping data: {e}")

finally:
    driver.quit()


                                               Company Earnings Date     Col 3 Col 4 Col 5 Col 6         Col 7 Col 8       Col 9      Col 10         Symbol
0                                 AVGOD\nBROADCOM INC.  736.309 BUSD   1.21USD     —     —     —    12.97 BUSD     —  2024-09-05  2024-07-31   After Market
1                                  CPRTD\nCOPART, INC.   51.204 BUSD   0.37USD     —     —     —    1.074 BUSD     —  2024-09-04  2024-07-31       Intraday
2                            KRD\nKROGER COMPANY (THE)   38.443 BUSD   0.91USD     —     —     —   34.082 BUSD     —  2024-09-06  2024-07-31       Intraday
3                                     ZSD\nZSCALER INC   29.318 BUSD   0.70USD     —     —     —  567.465 MUSD     —  2024-09-03  2024-07-31   After Market
4             HPED\nHEWLETT PACKARD ENTERPRISE COMPANY   24.694 BUSD   0.47USD     —     —     —    7.659 BUSD     —  2024-09-04  2024-07-31   After Market
5                                   IOTD\nSAMSARA INC.   22.721 

In [60]:
##### try optimizing 

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
driver.get("https://tradingview.com/markets/stocks-usa/earnings/")

# Click on the "Next Week" button
driver.find_element(By.XPATH, "//div[.='Next Week']").click()

# Shorter sleep time
time.sleep(3)

try:
    # Collect the table rows
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    data = []
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) == 11:
            # Extract the cell text data directly
            company = cells[0].text
            earnings_date = cells[1].text
            col_3 = cells[2].text
            col_4 = cells[3].text
            col_5 = cells[4].text
            col_6 = cells[5].text
            col_7 = cells[6].text
            col_8 = cells[7].text
            col_9 = cells[8].text
            col_10 = cells[9].text
            
            # Determine the symbol from the last cell
            last_cell = cells[10]
            symbol = "After Market"  # Default value

            # Check for SVG elements to determine market time
            svg_elements = last_cell.find_elements(By.TAG_NAME, "svg")
            if svg_elements:
                paths = svg_elements[0].find_elements(By.TAG_NAME, "path")
                if len(paths) > 1:  # Example condition for "Before Market"
                    symbol = "Before Market"
            elif last_cell.find_elements(By.XPATH, ".//span[text()='—']"):
                symbol = "Intraday"

            # Append the data to the list
            data.append({
                "Company": company,
                "Earnings Date": earnings_date,
                "Col 3": col_3,
                "Col 4": col_4,
                "Col 5": col_5,
                "Col 6": col_6,
                "Col 7": col_7,
                "Col 8": col_8,
                "Col 9": col_9,
                "Col 10": col_10,
                "Symbol": symbol
            })
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    print(df)

except Exception as e:
    print(f"Error scraping data: {e}")

finally:
    driver.quit()

                                               Company Earnings Date     Col 3 Col 4 Col 5 Col 6         Col 7 Col 8       Col 9      Col 10         Symbol
0                                 AVGOD\nBROADCOM INC.  736.309 BUSD   1.21USD     —     —     —    12.97 BUSD     —  2024-09-05  2024-07-31   After Market
1                                  CPRTD\nCOPART, INC.   51.204 BUSD   0.37USD     —     —     —    1.074 BUSD     —  2024-09-04  2024-07-31       Intraday
2                            KRD\nKROGER COMPANY (THE)   38.443 BUSD   0.91USD     —     —     —   34.082 BUSD     —  2024-09-06  2024-07-31       Intraday
3                                     ZSD\nZSCALER INC   29.318 BUSD   0.70USD     —     —     —  567.465 MUSD     —  2024-09-03  2024-07-31   After Market
4             HPED\nHEWLETT PACKARD ENTERPRISE COMPANY   24.694 BUSD   0.47USD     —     —     —    7.659 BUSD     —  2024-09-04  2024-07-31   After Market
5                                   IOTD\nSAMSARA INC.   22.721 

In [49]:
import requests
from bs4 import BeautifulSoup

# URL of the TradingView earnings page
url = "https://www.tradingview.com/markets/stocks-usa/earnings/"

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with earnings data
table = soup.find('table', class_='tv-data-table')

# Extract table rows
rows = table.find_all('tr')

# List to store extracted data
data = []

# Iterate over each row
for row in rows:
    columns = row.find_all('td')
    row_data = []
    
    # Extract visible text from each column
    for col in columns:
        text = col.get_text(strip=True)
        row_data.append(text)
        
        # Check for hidden symbols (moon, dash, sun) within each column
        hidden_symbol = None
        if col.find('path', d="M8.36 3.608a6.002 6.002 0 1 0 2.755 11.672 6.01 6.01 0 0 0 4.277-4.64 5 5 0 1 1-7.03-7.03z"):
            hidden_symbol = "Moon"
        elif col.find('span', class_='tv-screener-table__cell--color-normal'):
            hidden_symbol = "Dash"
        elif col.find('path', d="M13.5 8.5h3M.5 8.5h3M8.5 3.5v-3M13 4l2-2M2 15l2-2"):
            hidden_symbol = "Sun"
        
        if hidden_symbol:
            row_data.append(hidden_symbol)  # Add the hidden symbol to the row data
    
    data.append(row_data)

df=pd.DataFrame(data)

# Print extracted data
# for entry in data:
#     print(entry)

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the TradingView earnings page
url = "https://www.tradingview.com/markets/stocks-usa/earnings/"

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with earnings data
table = soup.find('table', class_='tv-data-table')

# Extract table rows
rows = table.find_all('tr')

# List to store extracted data
data = []

# Iterate over each row
for row in rows:
    columns = row.find_all('td')
    
    # Ensure that there are at least 11 columns
    if len(columns) >= 11:
        row_data = []
        
        # Extract visible text from the first 10 columns
        for col in columns[:-1]:
            text = col.get_text(strip=True)
            row_data.append(text)
        
        # Extract the hidden symbol from the last column
        hidden_symbol = None
        last_col = columns[-1]
        
        # Check for the moon, dash, or sun symbol in the last column
        if last_col.find('svg') and last_col.find('path', d="M8.36 3.608a6.002 6.002 0 1 0 2.755 11.672 6.01 6.01 0 0 0 4.277-4.64 5 5 0 1 1-7.03-7.03z"):
            hidden_symbol = "Moon"
        elif last_col.find('span', class_='tv-screener-table__cell--color-normal'):
            hidden_symbol = "Dash"
        elif last_col.find('svg') and last_col.find('path', d="M13.5 8.5h3M.5 8.5h3M8.5 3.5v-3M13 4l2-2M2 15l2-2"):
            hidden_symbol = "Sun"
        
        row_data.append(hidden_symbol)  # Add the hidden symbol to the row data
        
        data.append(row_data)

# Convert to pandas DataFrame
df = pd.DataFrame(data)

print(df)

                                                    0         1      2   \
0                               NVDANVIDIA Corporation    3.156T   0.65   
1                                  CRMSalesforce, Inc.  256.010B   2.35   
2                       CRWDCrowdStrike Holdings, Inc.   65.666B   0.97   
3                                           HPQHP Inc.   34.837B   0.86   
4                               VEEVVeeva Systems Inc.   32.371B   1.53   
..                                                 ...       ...    ...   
145             ELMSQElectric Last Mile Solutions Inc.    8.682M      —   
146  BNOXBionomics Limited - American Depository Sh...    8.272M  -0.95   
147              ADNAdvent Technologies Holdings, Inc.    8.268M      —   
148                         NLSPNLS Pharmaceutics Ltd.    8.073M      —   
149                        KEGXKEY ENERGY SERVICES INC    7.580M      —   

        3     4     5         6         7           8           9     10  
0     0.61  0.05  9.30  

In [42]:
#################################################
# 
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the TradingView earnings page
url = "https://www.tradingview.com/markets/stocks-france/earnings/"


# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with earnings data
table = soup.find('table', class_='tv-data-table')

# Extract table rows
rows = table.find_all('tr')

# List to store extracted data
data = []

# Iterate over each row
for row in rows:
    columns = row.find_all('td')
    
    # Ensure that there are at least 11 columns
    if len(columns) >= 11:
        row_data = []
        
        # Extract visible text from the first 10 columns
        for col in columns[:-1]:
            text = col.get_text(strip=True)
            row_data.append(text)
        
        # Extract the hidden symbol from the last column
        hidden_symbol = None
        last_col = columns[-1]
        
        # Check for the moon symbol
        if last_col.find('svg') and last_col.find('path', d="M8.36 3.608a6.002 6.002 0 1 0 2.755 11.672 6.01 6.01 0 0 0 4.277-4.64 5 5 0 1 1-7.03-7.03z"):
            hidden_symbol = "Moon"
        
        # Check for the sun symbol
        elif last_col.find('svg') and last_col.find('path', d="M13.5 8.5h3M.5 8.5h3M8.5 3.5v-3M13 4l2-2M2 15l2-2"):
            hidden_symbol = "Sun"
        
        # Check for the dash symbol (only if no SVG symbols were found)
        elif last_col.find('span', class_='tv-screener-table__cell--color-normal'):
            hidden_symbol = "Dash"
        
        row_data.append(hidden_symbol)  # Add the hidden symbol to the row data
        
        data.append(row_data)

# Convert to pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

                          0         1  2     3      4      5  6       7           8           9     10
0            RIPERNOD RICARD   32.449B  —  2.29  -0.07  -3.08  —  5.073B  2025-02-06  2024-12-30  Dash
1               GLOGL EVENTS  492.989M  —     —      —      —  —       —  2024-08-29  2024-06-29  Dash
2  ALVIOVALERIO THERAPEUTICS   12.634M  —     —      —      —  —       —  2024-08-29  2024-06-29  Dash


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the TradingView earnings page
url = "https://www.tradingview.com/markets/stocks-france/earnings/"

# Simulate a browser request by setting headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.tradingview.com/',
    'X-Requested-With': 'XMLHttpRequest'
}

# Send a GET request to the page with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables with earnings data
tables = soup.find_all('table', class_='tv-data-table')

# List to store extracted data from all tables
all_data = []

# Extract data from all found tables
for table in tables:
    rows = table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 11:
            row_data = [col.get_text(strip=True) for col in columns]
            all_data.append(row_data)

# Convert the extracted data to pandas DataFrame
df = pd.DataFrame(all_data)

# Print extracted table data
print("Extracted Table Data:")
print(df)

Extracted Table Data:
Empty DataFrame
Columns: []
Index: []


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the TradingView earnings page
url = "https://www.tradingview.com/markets/stocks-usa/earnings/"

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with earnings data
table = soup.find('table', class_='tv-data-table')

# Extract table rows
rows = table.find_all('tr')

# List to store extracted data
data = []

# Iterate over each row
for row in rows:
    columns = row.find_all('td')
    
    # Ensure that there are at least 11 columns
    if len(columns) >= 11:
        row_data = []
        
        # Extract visible text from the first 10 columns
        for col in columns[:-1]:
            text = col.get_text(strip=True)
            row_data.append(text)
        
        # Extract the hidden symbol from the last column
        hidden_symbol = None
        last_col = columns[-1]
        
        # Check if an SVG element exists
        svg = last_col.find('svg')
        
        if svg:
            # Convert the SVG content to a string for easier comparison
            svg_content = str(svg)
            
            # Check for the moon symbol
            if "M8.36 3.608a6.002 6.002" in svg_content:
                hidden_symbol = "Moon"
            # Check for the sun symbol based on the overall structure
            elif "<circle cx=\"8.5\" cy=\"8.5\" r=\"3\"></circle>" in svg_content and "<path stroke-linecap=\"square\"" in svg_content:
                hidden_symbol = "Sun"
        else:
            # If no SVG is present, assume it's a dash
            if last_col.find('span', class_='tv-screener-table__cell--color-normal'):
                hidden_symbol = "Dash"
        
        row_data.append(hidden_symbol)  # Add the hidden symbol to the row data
        
        data.append(row_data)

# Convert to pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

In [7]:
%reset